In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import nltk
import re

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import csv

In [3]:
df = pd.read_csv('/kaggle/input/medium-articles/medium_articles.csv')
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [5]:
df = df[['title', 'text']]
df.shape

(192368, 2)

In [6]:
df['text'] = df['text'].str.replace('\n', '</n/>')

sample_blog = df['text'][18]
print(sample_blog)

A Social Worker Offered Mormon Lingo to Me When I Was in Crisis, Told Me to Think Happy Thoughts, and Hung Up on Me — While I Was Still in Crisis Rhett Wilkinson Follow May 26, 2019 · 3 min read</n/></n/>I called a crisis line. You may say it was tragically less than helpful. In fact, it only made the situation worse. The issue was created by a social worker on the other end of the Utah County Crisis Line.</n/></n/>(United Way of Utah County)</n/></n/>Immediately after telling her I am considering killing myself, this person talked about “the other side.”</n/></n/>I’m not against talk about heaven, but “the other side” comes ripped from language used in culture of The Church of Jesus Christ of Latter-day Saints.</n/></n/>Which still would not be bad, except that she is not speaking organically. Not from the heart. When I was suicidal.</n/></n/>Further, she did nothing more than parrot what the system she is in fed her.</n/></n/>And that’s what she had to offer above anything else when 

In [7]:
sentences = sent_tokenize(sample_blog)
sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')

sentence_tokens = [[word for word in sentence.split(' ') if word not in stop_words] for sentence in sentences_clean]

w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)

sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
max_len = max([len(tokens) for tokens in sentence_tokens])

sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant') for embedding in sentence_embeddings]

In [8]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])

for i, row_embedding in enumerate(sentence_embeddings):
    for j, column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph, max_iter=600)

top_sentence = {sentence:scores[index] for index, sentence in enumerate(sentences)}

sentNeeded = round(0.25 * len(sentences)) - 1
top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
summary = ""
for sent in sentences:
    if sent in top.keys():
        summary += sent
print(summary)

When I was suicidal.</n/></n/>Further, she did nothing more than parrot what the system she is in fed her.</n/></n/>And that’s what she had to offer above anything else when I wanted nothing more than to die.</n/></n/>Then she went on to talk about thinking positive thoughts.I do know social work well enough that least to know that simply thinking positive thoughts doesn’t help anyone depressed, let alone suicidal.Her response?</n/></n/>Hang up.</n/></n/>On someone who may kill themselves?</n/></n/>And she said she has been a social worker for 15 years.</n/></n/>How often, then, did she miss out on helping folks in real pain?


In [9]:
count = 0
def generateSummary(blog):
    global count
    count +=1
    print("Summarizing blog ",count)
    try:
        sentences = sent_tokenize(blog)
        sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in sentences]
        stop_words = stopwords.words('english')

        sentence_tokens = [[word for word in sentence.split(' ') if word not in stop_words] for sentence in sentences_clean]

        w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)

        sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
        max_len = max([len(tokens) for tokens in sentence_tokens])

        sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant') for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])

        for i, row_embedding in enumerate(sentence_embeddings):
            for j, column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph, max_iter=600)

        top_sentence = {sentence:scores[index] for index, sentence in enumerate(sentences)}

        sentNeeded = round(0.25 * len(sentences)) - 1
        top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:sentNeeded])
        summary = ""
        for sent in sentences:
            if sent in top.keys():
                summary += sent
        return summary
    except:
        return float("NaN")

In [ ]:
import math

filename = "articlesSet.csv"
fields = ['title', 'summary', 'content']

# writing to csv file
with open(filename, 'a') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(fields)

    def callback(row):
        summary = generateSummary(row['text'])
        if type(summary) is str:
            return
        rows = [row['title'],summary, row['text']]
        csvwriter.writerow(rows)

    df.apply(callback, axis=1)


Summarizing blog  1
Summarizing blog  2
Summarizing blog  3
Summarizing blog  4
Summarizing blog  5
Summarizing blog  6
Summarizing blog  7
Summarizing blog  8
Summarizing blog  9
Summarizing blog  10
Summarizing blog  11
Summarizing blog  12
Summarizing blog  13
Summarizing blog  14
Summarizing blog  15
Summarizing blog  16
Summarizing blog  17
Summarizing blog  18
Summarizing blog  19
Summarizing blog  20
Summarizing blog  21
Summarizing blog  22
Summarizing blog  23


/opt/conda/lib/python3.10/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Summarizing blog  24
Summarizing blog  25
Summarizing blog  26
Summarizing blog  27
Summarizing blog  28
Summarizing blog  29
Summarizing blog  30
Summarizing blog  31
Summarizing blog  32
Summarizing blog  33
Summarizing blog  34
Summarizing blog  35
Summarizing blog  36
Summarizing blog  37
Summarizing blog  38
Summarizing blog  39
Summarizing blog  40
Summarizing blog  41
Summarizing blog  42
Summarizing blog  43
Summarizing blog  44
Summarizing blog  45
Summarizing blog  46
Summarizing blog  47
Summarizing blog  48
Summarizing blog  49
Summarizing blog  50
Summarizing blog  51
Summarizing blog  52
Summarizing blog  53
Summarizing blog  54
Summarizing blog  55
Summarizing blog  56
Summarizing blog  57
Summarizing blog  58
Summarizing blog  59
Summarizing blog  60
Summarizing blog  61
Summarizing blog  62
Summarizing blog  63
Summarizing blog  64
Summarizing blog  65
Summarizing blog  66
Summarizing blog  67
Summarizing blog  68
Summarizing blog  69
Summarizing blog  70
Summarizing b